# Measuring spectral properties within MCXC $R_{500}$ apertures

To provide context to our own measurements of $R_{500}$ (and more importantly the global properties we measure within them) we shall measure global $T_{\rm{X}}$ and $L_{\rm{X}}$ within apertures defined by the original MCXC values of $R_{500}$. The same XMM data utilised for our fiducial measurements of temperature and luminosity (and the same analysis methods) will be employed.

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 50
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sources import BaseSource
from xga.xspec import single_temp_apec
from xga.exceptions import ModelNotAssociatedError

# This is a bit cheeky, but suppresses the warnings that XGA spits out (they are 
#  useful, but not when I'm trying to present this notebook on GitHub)
import warnings
warnings.filterwarnings('ignore')

# Set up a variable that controls how long individual XSPEC fits are allowed to run
timeout = Quantity(6, 'hr')

%matplotlib inline

## Defining the cosmology

We make use of a concordance LambdaCDM model:

In [2]:
cosmo = LambdaCDM(70, 0.3, 0.7)

## Reading in the sample

We need the information from our sample file to declare the XGA ClusterSample further down, primarily the location and redshift, but we also make use of the MCXC estimate of $R_{500}$ to get some idea of the scale of these objects, even if we don't fully trust that measurement:

In [3]:
samp = pd.read_csv('../../sample_files/X-LoVoCCSI.csv')
samp

,LoVoCCSID,Name,start_ra,start_dec,MCXC_Redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec,MCXC_Lx500_0.1_2.4
0,1,A2029,227.734300,5.745471,0.0766,1.3344,227.73000,5.720000,227.734300,5.745471,8.726709e+44
1,2,A401,44.740000,13.580000,0.0739,1.2421,44.74000,13.580000,NaN,NaN,6.088643e+44
2,4A,A85North,10.458750,-9.301944,0.0555,1.2103,10.45875,-9.301944,NaN,NaN,5.100085e+44
3,4B,A85South,10.451487,-9.460007,0.0555,1.2103,10.45875,-9.301944,10.451487,-9.460007,5.100085e+44
4,5,A3667,303.157313,-56.845978,0.0556,1.1990,303.13000,-56.830000,303.157313,-56.845978,4.871933e+44
...,...,...,...,...,...,...,...,...,...,...,...
62,121,A3128,52.466189,-52.580728,0.0624,0.8831,52.50000,-52.600000,52.466189,-52.580728,1.101682e+44
63,122,A1023,157.000000,-6.800000,0.1176,0.8553,157.00000,-6.800000,NaN,NaN,1.095941e+44
64,123,A3528,193.670000,-29.220000,0.0544,0.8855,193.67000,-29.220000,NaN,NaN,1.093054e+44
65,131,A761,137.651250,-10.581111,0.0916,0.8627,137.65125,-10.581111,NaN,NaN,1.063423e+44


## Setting up an XGA ClusterSample

We set up an XGA ClusterSample object by passing in the information for the entire LoVoCCS sample, as XGA will determine which of the objects actually has XMM data available for itself. We also use the MCXC estimate of $R_{500}$ to 'clean' the observations that have been retrieved for each object, specifying that 10% of the $R_{500}$ region of each cluster must fall on a given observation for that observation to be used; this cleaning threshold is often set higher, but for this population of large, low-redshift, objects we use this smaller fraction to include offset observations of the outskirts. 

In [4]:
ra = samp['start_ra'].values
dec = samp['start_dec'].values
z = samp['MCXC_Redshift'].values
r500 = Quantity(samp['MCXC_R500'].values, 'Mpc')
name = samp['LoVoCCSID'].apply(lambda x: 'LoVoCCS-' + str(x)).values

srcs = ClusterSample(ra, dec, z, name, r500=r500, clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.9, 
                     use_peak=False, load_fits=True)

Generating products of type(s) expmap: 100%|██████████| 27/27 [00:00<00:00, 27.24it/s]


This dictionary shows the names of those clusters which are not included in the ClusterSample because they lack data:

In [5]:
srcs.failed_reasons

{'LoVoCCS-55': 'NoMatch',
 'LoVoCCS-108': 'NoMatch',
 'LoVoCCS-122': 'NoMatch',
 'LoVoCCS-1': 'Failed ObsClean',
 'LoVoCCS-2': 'Failed ObsClean',
 'LoVoCCS-4A': 'Failed ObsClean',
 'LoVoCCS-4B': 'Failed ObsClean',
 'LoVoCCS-5': 'Failed ObsClean',
 'LoVoCCS-9': 'Failed ObsClean',
 'LoVoCCS-11': 'Failed ObsClean',
 'LoVoCCS-12': 'Failed ObsClean',
 'LoVoCCS-21': 'Failed ObsClean',
 'LoVoCCS-27': 'Failed ObsClean',
 'LoVoCCS-30': 'Failed ObsClean',
 'LoVoCCS-33': 'Failed ObsClean',
 'LoVoCCS-41B': 'Failed ObsClean',
 'LoVoCCS-41C': 'Failed ObsClean',
 'LoVoCCS-46B': 'Failed ObsClean',
 'LoVoCCS-48A': 'Failed ObsClean',
 'LoVoCCS-48B': 'Failed ObsClean',
 'LoVoCCS-65': 'Failed ObsClean',
 'LoVoCCS-66A': 'Failed ObsClean',
 'LoVoCCS-67': 'Failed ObsClean',
 'LoVoCCS-90': 'Failed ObsClean',
 'LoVoCCS-93A': 'Failed ObsClean',
 'LoVoCCS-93B': 'Failed ObsClean',
 'LoVoCCS-123': 'Failed ObsClean'}

In [6]:
srcs.info()


-----------------------------------------------------
Number of Sources - 40
Redshift Information - True
Sources with ≥1 detection - 37 [92%]
-----------------------------------------------------



## Generating and fitting spectra within MCXC measurements of $R_{500}$

We use XGA to generate spectra within apertures defined by the MCXC-measured $R_{500}$ value, which are then fit with an absorbed APEC plasma emission model.

### Single-temperature absorbed APEC model

In [7]:
single_temp_apec(srcs, 'r500', freeze_met=False, one_rmf=False, timeout=timeout)

Running XSPEC Fits: 100%|██████████| 40/40 [07:28<00:00, 11.21s/it]  


This is a quick way to check how many successful fits with this model to this aperture's spectra there were:

In [8]:
print(len(np.where(~np.isnan(srcs.Tx('r500', quality_checks=False)[:, 0]))[0]))

39


## Generating and fitting spectra within MCXC measurement of 0.15-1$R_{500}$

Core-excised properties are often very useful, and can have lower scatter with cluster mass than core-included properties, so we measure them as well.

### Single-temperature absorbed APEC model

In [9]:
single_temp_apec(srcs, 'r500', srcs.r500*0.15, freeze_met=False, one_rmf=False, timeout=timeout)

Running XSPEC Fits: 100%|██████████| 40/40 [09:58<00:00, 14.96s/it]


This is a quick way to check how many successful fits with this model to this aperture's spectra there were:

In [11]:
print(len(np.where(~np.isnan(srcs.Tx('r500', inner_radius=srcs.r500*0.15, quality_checks=False)[:, 0]))[0]))

39


## Saving results 

In [43]:
met = []
metce = []
for src_ind, src in enumerate(srcs):
    try:
        met.append(src.get_results('r500', 'constant*tbabs*apec', par='Abundanc'))
    except ModelNotAssociatedError:
        met.append(np.array([np.NaN, np.NaN, np.NaN]))

    try:
        metce.append(src.get_results('r500', 'constant*tbabs*apec', par='Abundanc', inner_radius=srcs.r500[src_ind]*0.15))
    except ModelNotAssociatedError:
        metce.append(np.array([np.NaN, np.NaN, np.NaN]))

met = np.array(met).round(3)
metce = np.array(metce).round(3)

dat = np.concatenate([srcs.names[..., None], 
                      srcs.nHs[..., None].value, 
                      srcs.Tx('r500', quality_checks=False).value.round(3), 
                      met,
                      srcs.Lx('r500', quality_checks=False).to('1e+44 erg/s').value.round(3), 
                      srcs.Lx('r500', lo_en=Quantity(0.01, 'keV'), hi_en=Quantity(100.0, 'keV'), 
                              quality_checks=False).to('1e+44 erg/s').value.round(3),
                      srcs.Tx('r500', inner_radius=srcs.r500*0.15, quality_checks=False).value.round(3), 
                      metce,
                      srcs.Lx('r500', inner_radius=srcs.r500*0.15, quality_checks=False).to('1e+44 erg/s').value.round(3), 
                      srcs.Lx('r500', inner_radius=srcs.r500*0.15, lo_en=Quantity(0.01, 'keV'), hi_en=Quantity(100.0, 'keV'), 
                              quality_checks=False).to('1e+44 erg/s').value.round(3)], axis=1)

cols = ['name', 'nH', 'Tx500', 'Tx500-', 'Tx500+', 'Zmet500', 'Zmet500-', 'Zmet500+', 'Lx500_0.5-2.0', 'Lx500_0.5-2.0-', 
        'Lx500_0.5-2.0+', 'Lx500_0.01-100.0', 'Lx500_0.01-100.0-', 'Lx500_0.01-100.0+', 
        'Tx500ce', 'Tx500ce-', 'Tx500ce+', 'Zmet500ce', 'Zmet500ce-', 'Zmet500ce+', 'Lx500ce_0.5-2.0', 'Lx500ce_0.5-2.0-', 
        'Lx500ce_0.5-2.0+', 'Lx500ce_0.01-100.0', 'Lx500ce_0.01-100.0-', 'Lx500ce_0.01-100.0+']

apec_res = pd.DataFrame(dat, columns=cols)
apec_res = apec_res.astype({col: float for col in apec_res.columns[1:]})

In [44]:
apec_res.to_csv("../../outputs/results/mcxc_r500_txlx_metfree.csv", index=False)
apec_res

,name,nH,Tx500,Tx500-,Tx500+,Zmet500,Zmet500-,Zmet500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+,Tx500ce,Tx500ce-,Tx500ce+,Zmet500ce,Zmet500ce-,Zmet500ce+,Lx500ce_0.5-2.0,Lx500ce_0.5-2.0-,Lx500ce_0.5-2.0+,Lx500ce_0.01-100.0,Lx500ce_0.01-100.0-,Lx500ce_0.01-100.0+
0,LoVoCCS-7,0.02770,5.912,0.063,0.063,0.243,0.015,0.015,3.022,0.010,0.010,10.637,0.067,8.200000e-02,5.697,0.082,0.090,0.206,0.021,0.021,2.023,0.010,0.010,6.978,0.052,0.064
1,LoVoCCS-10,0.01530,5.576,0.089,0.088,0.287,0.026,0.026,3.007,0.015,0.015,10.325,0.099,1.200000e-01,5.449,0.132,0.143,0.173,0.036,0.037,1.890,0.013,0.014,6.367,0.096,0.099
2,LoVoCCS-13,0.01120,3.396,0.039,0.039,0.312,0.013,0.013,2.476,0.015,0.016,6.957,0.062,4.500000e-02,2.641,0.082,0.085,0.096,0.017,0.018,0.868,0.018,0.016,2.243,0.055,0.050
3,LoVoCCS-14,0.09900,5.280,0.278,0.317,0.174,0.087,0.063,2.091,0.048,0.027,6.916,0.181,2.180000e-01,5.154,0.359,0.421,0.108,0.108,0.087,1.625,0.037,0.039,5.303,0.213,0.146
4,LoVoCCS-15,0.02240,3.175,0.011,0.011,0.246,0.004,0.004,2.602,0.008,0.006,6.917,0.017,2.300000e-02,2.959,0.026,0.026,0.106,0.007,0.007,0.907,0.006,0.005,2.346,0.019,0.019
5,LoVoCCS-18,0.01280,4.683,0.038,0.038,0.200,0.011,0.011,2.634,0.007,0.008,8.287,0.036,3.800000e-02,4.302,0.056,0.060,0.089,0.015,0.015,1.557,0.006,0.007,4.683,0.025,0.039
6,LoVoCCS-22,0.02930,4.250,0.083,0.091,0.091,0.023,0.023,1.930,0.011,0.010,5.778,0.073,6.900000e-02,4.513,0.114,0.125,0.126,0.031,0.032,1.469,0.011,0.011,4.523,0.073,0.044
7,LoVoCCS-24,0.02190,4.871,0.048,0.050,0.202,0.013,0.013,2.152,0.007,0.009,6.897,0.035,3.700000e-02,4.736,0.056,0.056,0.166,0.015,0.016,1.637,0.007,0.007,5.168,0.025,0.042
8,LoVoCCS-26,0.04220,4.649,0.077,0.077,0.336,0.025,0.026,2.134,0.013,0.011,6.749,0.050,7.600000e-02,4.847,0.127,0.139,0.276,0.037,0.039,1.301,0.010,0.010,4.180,0.075,0.041
9,LoVoCCS-28,0.01660,5.017,0.081,0.081,0.141,0.021,0.021,1.835,0.008,0.006,5.933,0.045,6.800000e-02,4.790,0.101,0.107,0.105,0.025,0.026,1.370,0.008,0.007,4.328,0.050,0.045
